In [0]:
%run Pre_ini

In [ ]:
#%run Get_syn.ipynb

In [ ]:
with open('syn.pickle','rb') as f:
    syn=pickle.load(f)

In [0]:
%run Worker #This could take some time

In [0]:
%run BM25F #This take about 16 gb RAM

In [0]:
with open('super_query_ext.pickle','rb') as f:
    query_ext=pickle.load(f)

with open('IDF_lem.pickle','rb') as f:
    IDF_lem=pickle.load(f)

In [ ]:
%run Text_passage #This could take forever

In [10]:
all_scores=defaultdict(list)
Passage=namedtuple('Passage',['sent','score_bert','pos','doc_id'])
for file_name in ['divide_cos0.txt','divide_cos1.txt','divide_cos2.txt','divide_cos3.txt']:
    with open(file_name,'r') as f:
        i=0
        for line in f:
            lines=line.strip('\n').split('\t')
            doc_id=lines[0]

            query=lines[1]
            lines=lines[2:]

            query_id=get_query_id[query]
            for line in lines:
                sent,score,score_pos=line.split(',')
                all_scores[query_id].append(Passage(sent,1-float(score),float(score_pos),int(doc_id)))
            i+=1
        print(i)

10039
10161
10210
8362


In [0]:
%run Passage_full.ipynb

In [ ]:
#%run Text_desm_fast.ipynb #This could take forever

In [ ]:
#%run Text_desm_word2vec.ipynb  #This could take even longer

In [ ]:
%run DESM.ipynb

In [0]:
with open('queries.fixed.txt','r',encoding='utf-8-sig') as f:
    with open('Passage.txt','w') as out:
        i=0
        for line in f:
            query_id=line.strip('\n').split('\t')[0]
            
            tmp=full(int(query_id),all_scores)

            out.write(query_id+'\t')
            for x in tmp:
                doc_id=x.doc_id
                score=1/(1/(x.score_bert+1e-5) + 1/(x.score_idf +1e-5))+0.15*x.pos
                out.write(str(score)+','+str(doc_id))
                out.write(' ')
            out.write('\n')
            i+=1
            print(i)

In [0]:
def get_bm():
    with open('BM25.txt','r') as f:
        ans=dict()
        for line in f:
            query_id,s=line.strip().split('\t')
            query_id=int(query_id)

            score_bm=defaultdict(float)
            for score in s.split():
                score,doc_id=score.split(',')
                score_bm[int(doc_id)]=float(score)
            ans[query_id]=score_bm
        return ans

In [0]:
def get_passage():
    with open('Passage.txt','r') as f:
        ans=dict()
        for line in f:
            query_id,s=line.strip().split('\t')
            query_id=int(query_id)

            score_bm=defaultdict(float)
            for score in s.split():
                score,doc_id=score.split(',')
                score_bm[int(doc_id)]=float(score)
            ans[query_id]=score_bm
        return ans

In [0]:
def get_desm():
    with open('Desm.txt','r') as f:
        ans=dict()
        for line in f:
            query_id,s=line.strip().split('\t')
            query_id=int(query_id)

            score_bm=defaultdict(float)
            for score in s.split():
                score,doc_id=score.split(',')
                score_bm[int(doc_id)]=float(score)
            ans[query_id]=score_bm
        return ans

In [0]:
bm_score=get_bm()
passage_score=get_passage()
desm_score=get_desm()

ans='QueryId,DocumentId\n'

alpha1=0.00125
alpha2=1
alpha3=5

for query_id in range(1,len(get_query)+1):
    scores=dict()
    for doc_id in candidates[query_id]:
        scores[doc_id]=(bm_score[query_id][doc_id]*alpha1+
                        passage_score[query_id][doc_id]*alpha2+
                        desm_score[query_id][doc_id]*alpha3)
    scores=sorted(scores.items(),key=lambda x:-x[1])[:10]
    ans=reduce(lambda ans,score:ans+str(query_id)+','+str(score[0])+'\n',scores,ans)

In [0]:
with open('ans.txt','w') as out:
    out.write(ans)